## Предварительная обработка

### 1. Загрузите и прочитайте CSV-файл и сделайте ID индексным столбцом

In [1]:
import pandas as pd

df = pd.read_csv('../data/auto.csv', sep=',', index_col=0)
print(df)

        CarNumber    Make_n_model  Refund    Fines  History
ID                                                         
0    Y163O8161RUS      Ford Focus     2.0   3200.0      NaN
1     E432XX77RUS    Toyota Camry     1.0   6500.0      NaN
2     7184TT36RUS      Ford Focus     1.0   2100.0      NaN
3    X582HE161RUS      Ford Focus     2.0   2000.0      NaN
4    E34877152RUS      Ford Focus     2.0   6100.0      NaN
..            ...             ...     ...      ...      ...
926  Y163O8161RUS      Ford Focus     2.0   1600.0      NaN
927  M0309X197RUS      Ford Focus     1.0  22300.0      NaN
928  O673E8197RUS      Ford Focus     2.0    600.0      NaN
929  8610T8154RUS      Ford Focus     1.0   2000.0      NaN
930  H419XE197RUS  Toyota Corolla     2.0      NaN      2.0

[931 rows x 5 columns]


### 2. Подсчитать количество наблюдений с помощью метода count()

In [2]:
print(df.count())

CarNumber       931
Make_n_model    931
Refund          914
Fines           869
History          82
dtype: int64


### 3. Удалить дубликаты, принимая во внимание только следующие столбцы: CarNumber, Make_n_model, Fines
- из двух равных наблюдений нужно выбрать последнее
- проверьте количество наблюдений еще раз

In [3]:
df=df.drop_duplicates(subset=['CarNumber', 'Make_n_model', 'Fines'], keep = 'last') # из двух равных последнее
print(f"Количество наблюдений после удаления дубликатов: {len(df)}\n")
print(df)

Количество наблюдений после удаления дубликатов: 725

        CarNumber    Make_n_model  Refund    Fines  History
ID                                                         
0    Y163O8161RUS      Ford Focus     2.0   3200.0      NaN
1     E432XX77RUS    Toyota Camry     1.0   6500.0      NaN
2     7184TT36RUS      Ford Focus     1.0   2100.0      NaN
3    X582HE161RUS      Ford Focus     2.0   2000.0      NaN
5    92918M178RUS      Ford Focus     1.0   5700.0      NaN
..            ...             ...     ...      ...      ...
926  Y163O8161RUS      Ford Focus     2.0   1600.0      NaN
927  M0309X197RUS      Ford Focus     1.0  22300.0      NaN
928  O673E8197RUS      Ford Focus     2.0    600.0      NaN
929  8610T8154RUS      Ford Focus     1.0   2000.0      NaN
930  H419XE197RUS  Toyota Corolla     2.0      NaN      2.0

[725 rows x 5 columns]


### 4. Работа с пропущенными значениями
- проверьте, сколько значений отсутствует в каждом столбце
- удалите все столбцы с более чем 500 пропущенными значениями, используя аргумент thresh, проверьте, сколько пропущенных значений в каждом столбце
- замените все отсутствующие значения в столбце «Возврат» предыдущим значением в этом столбце для этой ячейки, используйте метод аргумента, проверьте, сколько значений отсутствует в каждом столбце
- замените все отсутствующие значения в столбце «Штрафы» средним значением этого столбца (исключите значения NA/NULL при вычислении среднего значения), проверьте, сколько значений отсутствует в каждом столбце

In [4]:
df = pd.read_csv('../data/auto.csv', na_values=[''])
missing_values = df.isna().sum()
print(f"Количество отсутствующих значений в каждом столбце: \n{missing_values}\n")

df = df.dropna(thresh=len(df) - 500, axis=1) # удаляем все столбцы с более чем 500 пропущенными значениями
# dropna() - метод pandas, который используется для удаления строк или столбцов с пропущенными значениями.
# thresh указывает минимальное количество ненулевых значений, которое должно быть в строке или столбце, чтобы она/он не был удален. 
# В данном случае thresh=len(df) - 500 означает, что столбец будет удален, если в нем более 500 пропущенных значений.
# axis=1 указывает, что операция должна применяться к столбцам.
missing_values_after_drop = df.isna().sum()
print(f"Количество отсутствующих значений после удаления столбцов, где меньше 500 значений: \n{missing_values_after_drop}\n")

df['Refund'] = df['Refund'].ffill()
missing_values_after_refund_fill = df.isna().sum()
print(f"Количество отсутствующих значений после замены в Refund: \n{missing_values_after_refund_fill}\n")

df['Fines'] = df['Fines'].fillna(df['Fines'].mean())
missing_values_after_fines_fill = df.isna().sum()
print(f"А теперь каждому по штрафу:\nКоличество отсутствующих значений после замены в Fines: \n{missing_values_after_fines_fill}\n")




Количество отсутствующих значений в каждом столбце: 
ID                0
CarNumber         0
Make_n_model      0
Refund           17
Fines            62
History         849
dtype: int64

Количество отсутствующих значений после удаления столбцов, где меньше 500 значений: 
ID               0
CarNumber        0
Make_n_model     0
Refund          17
Fines           62
dtype: int64

Количество отсутствующих значений после замены в Refund: 
ID               0
CarNumber        0
Make_n_model     0
Refund           0
Fines           62
dtype: int64

А теперь каждому по штрафу:
Количество отсутствующих значений после замены в Fines: 
ID              0
CarNumber       0
Make_n_model    0
Refund          0
Fines           0
dtype: int64



### 5.Разделите и проанализируйте марку и модель
- используйте метод apply как для разделения, так и для извлечения значений в новые столбцы Марка и Модель
- удалить столбец Make_n_model
- сохраните фрейм данных в JSON-файле auto.json

In [5]:
# apply() применяет функцию ко всем элементам столбца
df['Make'] = df['Make_n_model'].apply(lambda x: x.split(' ')[0])
df['Model'] = df['Make_n_model'].apply(lambda x: ' '.join(x.split(' ')[1:]))
df = df.drop(columns=['Make_n_model'])
df = df[['CarNumber', 'Refund', 'Fines', 'Make', 'Model']]
df.to_json('../data/auto.json', orient='records', indent=4)

print("Данные разделены по маркам и моделям авто и сохранены в auto.json")

Данные разделены по маркам и моделям авто и сохранены в auto.json


In [2]:
df['Fines'].mean()

NameError: name 'df' is not defined